In [1]:
import pandas as pd
import pickle
import requests
import json
import re
from collections import OrderedDict

from bs4 import BeautifulSoup

In [2]:
pickles = ['멘탈_관리_심리_탐구?q=g','문화·예술?q=g','건축·설계?q=g'
              ,'인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
                 '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

In [3]:
writer_list = []
for file in pickles:
    print(file)
    with open(file[:-4]+"_userId.txt","rb") as fr:
        writers = pickle.load(fr)
    writer_list.append(writers)

멘탈_관리_심리_탐구?q=g
문화·예술?q=g
건축·설계?q=g
인문학·철학?q=g
쉽게_읽는_역사?q=g
우리집_반려동물?q=g
멋진_캘리그래피?q=g
사랑·이별?q=g
감성_에세이?q=g


In [4]:
## 게시글 크롤링

data = []
json_data = {}
# res_text = []

def def_craw(writer_list,input_catenum):
    global json_data
    global data
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for i,url in enumerate(writer_list[input_catenum]):
#         print("전체%d개의 게시글 중 %d번째 글 진행중" %(len(writer_list[input_catenum]),i+1))
        if res_text == []: # 첫 시작에러 방지
            pass
        else :
            
            # to json
            json_data['title'] = tag_title  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
#         print(data)
        
        json_data = {} # 누적방지 초기화
        tmp_keyword = [] # 누적방지 초기화
        res_text = [] # 누적방지 초기화 
        
        # beautifulsoup
        html = requests.get('https://brunch.co.kr{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        
        if soup.find('title').text == "brunch":
            pass
        else:
            tag_title = soup.find('title').text # 해당페이지의 Title
            tag_url = soup.find("meta",property='og:url')['content'] # 본주소
            tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
            tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
            tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
            tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
            tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
            tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
            tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
            tag_url_meta = soup.find("meta",property='ks:richscrap')['content']
        
            to_json = json.loads(tag_url_meta)
            for i in to_json['cards']:
                if i['type'] == 'text':
                    res_text.append(i['body']['content'])

            if tag_like == None:
                tag_like = "0"
            else:
                tag_like = tag_like.text # 좋아요 수

            if tag_share == None:
                tag_share == "0"
            else:
                tag_share = tag_share.text # 공유 수

            if tag_comment == None:
                tag_comment =="0"
            else:
                tag_comment = tag_comment.text

            for keyword in tag_keyword:
                tmp_keyword.append(keyword.text)
    return data

In [ ]:
# '지구한바퀴_세계여행','그림·웹툰','시사·이슈','IT_트렌드','사진·촬영','',
#                  '취향저격_영화_리뷰','오늘은_이런_책','뮤직_인사이드','글쓰기_코치','직장인_현실_조언'
#                  ,'스타트업_경험담','육아_이야기','요리·레시피','건강·운동',

In [6]:


categories = ['멘탈_관리_심리_탐구','문화·예술','건축·설계','인문학·철학','쉽게_읽는_역사',
              '우리집_반려동물','멋진_캘리그래피','사랑·이별','감성_에세이']

for idx,writer in enumerate(writer_list):
    data = def_craw(writer_list,idx)
    
    del data[0]
    del data[0]
    to_json = OrderedDict()
    full_data = OrderedDict()
    
    to_json['name'] = categories[idx]
    to_json['version'] = "2020-04-01"
    to_json['data'] = data
    
    with open(categories[idx]+".json","w",encoding='utf-8') as make_file:
        json.dump(to_json,make_file,ensure_ascii=False, indent="\t")